Download ECOSTRESS data using NASA AρρEEARS api

In [1]:
import requests as r
import getpass, pprint, time, os, cgi, json

api = 'https://appeears.earthdatacloud.nasa.gov/api/'  # Set the AρρEEARS API to a variable

user = getpass.getpass(prompt = 'Enter NASA Earthdata Login Username: ')      # Input NASA Earthdata Login Username
password = getpass.getpass(prompt = 'Enter NASA Earthdata Login Password: ')  # Input NASA Earthdata Login Password

Enter NASA Earthdata Login Username:  ··········
Enter NASA Earthdata Login Password:  ···········


In [2]:
token_response = r.post('{}login'.format(api), auth=(user, password)).json() # Insert base API URL, call the login service, provide credentials & return json
del user, password                                                           # Remove user and password information upon a successful login 
token_response  

{'token_type': 'Bearer',
 'token': 'x-FNHQGf4hyeSzp0Xw8DhS4qag0G2MkJg0uL7rA_Y5sHVNZCzjaPJ_OxjiNfMrzXnJ1WHsncvQjrqtH2VGkRbA',
 'expiration': '2022-06-17T03:32:59Z'}

In [3]:
params = {'pretty': True}
token = token_response['token']                      # Save login token to a variable
head = {'Authorization': 'Bearer {}'.format(token)} 
tasks_response = r.get('{}task'.format(api), params=params, headers=head).json() # Query task service, setting params and header 
len(tasks_response)  ##

434

In [11]:
tasks_response[2]

{'error': None,
 'params': {'dates': [{'endDate': '12-31-2020',
    'recurring': False,
    'startDate': '01-01-2019',
    'yearRange': [1950, 2050]}],
  'layers': [{'layer': 'EVAPOTRANSPIRATION_PT_JPL_ETcanopy',
    'product': 'ECO3ETPTJPL.001'},
   {'layer': 'EVAPOTRANSPIRATION_PT_JPL_ETdaily',
    'product': 'ECO3ETPTJPL.001'}],
  'output': {'format': {'type': 'geotiff'}, 'projection': 'native'}},
 'status': 'done',
 'created': '2022-06-13T06:20:32.919137',
 'task_id': '42f8dedc-b1b3-4f42-9b7e-b263f02a30a1',
 'updated': '2022-06-13T09:07:47.236238',
 'user_id': '18075416868@163.com',
 'attempts': 1,
 'estimate': {'request_size': 359440835815.57117},
 'retry_at': None,
 'completed': '2022-06-13T09:07:47.235092',
 'has_swath': True,
 'task_name': '588',
 'task_type': 'area',
 'api_version': None,
 'svc_version': '3.4',
 'web_version': '3.4',
 'size_category': '0',
 'has_nsidc_daac': False,
 'expires_on': '2022-07-13T09:07:47.236238'}

In [13]:
inDir = 'G:/DataBase/MangroveGreenness/ClimateData/ET/ECOSTRESS/Grids/'    # IMPORTANT: Update to reflect directory on your OS
os.chdir(inDir) 
for i in [0,2]: ##
    task_name = tasks_response[i]['task_name']
    task_id = tasks_response[i]['task_id']

    destDir = os.path.join(inDir, task_name)                # Set up output directory using input directory and task name
    if os.path.exists(destDir):
        continue
    if not os.path.exists(destDir):
        os.makedirs(destDir)
    bundle = r.get('{}bundle/{}'.format(api,task_id), headers=head).json()  # Call API and return bundle contents for the task_id as json
    files = {}                                                       # Create empty dictionary
    for f in bundle['files']: files[f['file_id']] = f['file_name']
    for f in files:
        dl = r.get('{}bundle/{}/{}'.format(api, task_id, f), headers=head, stream=True, allow_redirects = 'True')                                # Get a stream to the bundle file
        if files[f].endswith('.tif'):
            filename = files[f].split('/')[1]
            filepath = os.path.join(destDir, filename)                                                       # Create output file path
            with open(filepath, 'wb') as f:                                                                  # Write file to dest dir
                for data in dl.iter_content(chunk_size=8192): f.write(data) 
    print('Downloaded files can be found at: {}'.format(destDir))

Downloaded files can be found at: G:/DataBase/MangroveGreenness/ClimateData/ET/ECOSTRESS/Grids/588
